In [2]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [3]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dwmatrix/matrix_two/predykcja_cen"

/content/drive/My Drive/Colab Notebooks/dwmatrix/matrix_two/predykcja_cen


In [0]:
df=pd.read_hdf('data/car.h5')

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list): continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values 

In [8]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  model = model
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return(np.mean(scores),np.std(scores))

In [24]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

In [25]:
a = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0), cat_feats)

(-18718.657185256638, 64.5424578125788)

In [26]:
xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[04:44:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:44:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:44:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [22]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50,learning_rate=0.1, seed=0)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

[04:36:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [0]:
feats = ['param_stan__cat','param_napęd__cat','param_rok-produkcji__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','param_faktura-vat__cat','param_moc__cat','param_pojemność-skokowa__cat','feature_kamera-cofania__cat','param_typ__cat','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_kod-silnika__cat','param_model-pojazdu__cat','param_wersja__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']


In [28]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[04:50:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:50:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:50:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13376.76526242052, 64.07943314577214)

In [30]:
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x:-1 if str(x) == 'None' else int(x))

In [32]:
feats = ['param_stan__cat','param_napęd__cat','param_rok-produkcji','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','param_faktura-vat__cat','param_moc__cat','param_pojemność-skokowa__cat','feature_kamera-cofania__cat','param_typ__cat','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_kod-silnika__cat','param_model-pojazdu__cat','param_wersja__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[04:54:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:55:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:55:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [35]:
feats = ['param_stan__cat','param_napęd__cat','param_rok-produkcji','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','param_faktura-vat__cat','param_moc','param_pojemność-skokowa__cat','feature_kamera-cofania__cat','param_typ__cat','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_kod-silnika__cat','param_model-pojazdu__cat','param_wersja__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[04:59:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:59:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:00:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9715.920237306504, 62.06177456495893)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))

In [37]:
feats = ['param_stan__cat','param_napęd__cat','param_rok-produkcji','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','param_faktura-vat__cat','param_moc','param_pojemność-skokowa','feature_kamera-cofania__cat','param_typ__cat','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_kod-silnika__cat','param_model-pojazdu__cat','param_wersja__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[05:04:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:04:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:04:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.051641194832, 72.75844333754988)